In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import functools, re
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from textblob import TextBlob
from pydub import AudioSegment

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Load the model
text_model =  tf.keras.models.load_model('/content/drive/MyDrive/Project/models/cyberbullying_model_bilstm.h5')
image_model =  tf.keras.models.load_model('/content/drive/MyDrive/Project/models/cyberbullying_model_bilstm_image.h5')
#load the dataset
df = pd.read_csv("/content/drive/MyDrive/Project/twitter.csv")
stopwords = set(stopwords.words("english"))
stopwords = [i.lower() for i in nltk.corpus.stopwords.words('english') + [chr(i) for i in range(97, 123)]]
x = df.tweet_text.apply(lambda text: re.sub("\s+", " ", ' '.join([i for i in re.sub("[^9A-Za-z ]", "" , re.sub("\\n", "", re.sub("\s+", " ", re.sub(r'http\S+', '', text.lower())))).split(" ") if i not in stopwords]))).values.astype(str)
y = pd.get_dummies(df.cyberbullying_type)
labels = list(y.columns)

In [ ]:
from tkinter import Tk, filedialog, Button, Label
import speech_recognition as sr
from PIL import Image
import cv2
import os
#from IPython.display import Image
from google.colab import files
from PIL import Image

def sentiment_analysis(text):
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 1
        else:
            return 0

def fun(arr):
        if arr in ['other_cyberbullying','gender','age','ethnicity','religion']:
            return 0
        else:
            return 1
def text_predict(custom_input):
    x=sentiment_analysis(custom_input)
    tokenizer=Tokenizer()

    # Preprocess the custom input text in the same way as you did for the training data
    custom_input = re.sub("\s+", " ", ' '.join([i for i in re.sub("[^9A-Za-z ]", "" , re.sub("\\n", "", re.sub("\s+", " ", re.sub(r'http\S+', '', custom_input.lower())))).split(" ") if i not in stopwords]))
    custom_input = tokenizer.texts_to_sequences([custom_input])
    custom_input = pad_sequences(custom_input, maxlen=100, padding='post', truncating='post')

    # Make the prediction using the model
    prediction = text_model.predict(custom_input)

    # Get the index of the highest predicted class
    predicted_class_index = np.argmax(prediction)

    # Get the corresponding label for the predicted class
    predicted_class = labels[predicted_class_index]

    # print("The predicted cyberbullying type is:", fun(predicted_class) or x )
    return fun(predicted_class)

while(1):
  choice = input("Enter choice:\n1.Text\n2.Image\n3.Audio\n4.Exit\n")    
  if choice == '1':
      custom_input=str(input("Enter text to check whether it is bully or not"))
      if text_predict(custom_input) == 0:
        print("The text is cyberbullying")
      else:
        print("The text is not cyber bullying")
      
  elif choice == '2':
      # Use files.upload() to upload an image file to Colab
      uploaded = files.upload()
      
      # Get the name of the uploaded file
      filename = next(iter(uploaded))

      # Load the uploaded image as a PIL image object
      img_pil = Image.open(filename)
      # Resize the image to (224, 224) using PIL's resize() function
      img_pil_resized = img_pil.resize((224, 224))

      # Convert the PIL image to a numpy array
      img_np = np.array(img_pil_resized)

      # Use cv2.resize() to resize the numpy array to (224, 224)
      img = cv2.resize(img_np, (224, 224))
      # Convert the image to a numpy array
      img = np.array([img])
      # Use the model to make a prediction
      prediction = image_model.predict(img)
      predicted_label = "Not Bullying" if prediction[0][0] > 0.5 else "Bullying"
      print(predicted_label)
  elif choice == '3':
      # Load the MP3 file
      filename=input("Enter filename")
      mp3_audio = AudioSegment.from_file("/content/drive/My Drive/Project/audios/"+filename+".mp3", format="mp3")

    # Save the MP3 file as a WAV fil
      mp3_audio.export("audio.wav", format="wav")

  # Transcribe the WAV file to text 
      r = sr.Recognizer()
      audio_file = sr.AudioFile("audio.wav")

      with audio_file as source:
          audio_text = r.record(source)

      result = r.recognize_google(audio_text, show_all=True)
      custom_input= result['alternative'][0]['transcript']
      print(custom_input)
      if text_predict(custom_input) == 1:
        print("The audio is cyberbullying")
      else:
        print("The audio is not cyber bullying")
  else:
    break

Enter choice:
1.Text
2.Image
3.Audio
4.Exit
1
Enter text to check whether it is bully or notYou are ugly
1/1 [==============================] - 0s 71ms/step
The text is cyberbullying
Enter choice:
1.Text
2.Image
3.Audio
4.Exit
2


Saving Non_bully (5).jpg to Non_bully (5) (1).jpg
1/1 [==============================] - 1s 510ms/step
Not Bullying
Enter choice:
1.Text
2.Image
3.Audio
4.Exit
3
Enter filenameaudio1
you are beautiful
1/1 [==============================] - 0s 46ms/step
The audio is not cyber bullying
Enter choice:
1.Text
2.Image
3.Audio
4.Exit
4
